In [199]:
# Import libraries

import simpy
import random

# Set Simulation Global Variables
PASSENGER_INTERVAL = 5
NUM_FLOORS = 8
#NUM_ELEVATORS = 4
MAX_CAPACITY = 4
MINUTES_IN_DAY = 24 * 60 # Total simulation time in minutes
SIM_TIME = MINUTES_IN_DAY # Total simulation time in minutes

In [200]:
def get_arrival_rate(current_time_minutes):
    
    """
    Returns the arrival rate (passengers per minute) based on time of day.
    Peak times:
    - 7:30-8:30 AM (450-510 minutes)
    - 12:00-1:00 PM (720-780 minutes) 
    - 5:00-10:00 PM (1020-1320 minutes)
    """
    
    # Convert time to minutes from midnight
    time_of_day = current_time_minutes % 1440
    
    # Morning rush: 7:30-8:30 AM
    if 450 <= time_of_day <= 510:
        return 1  # High arrival rate (1 passenger every 1 minutes on average)
    
    # Lunch rush: 12:00-1:00 PM
    elif 720 <= time_of_day <= 780:
        return 1  # High arrival rate (1 passenger every 1 minutes on average)
    
    # Evening rush: 5:00-10:00 PM
    elif 1020 <= time_of_day <= 1320:
        return 1  # Medium-high arrival rate (1 passenger every 1 minutes on average)
    
    # Off-peak hours
    else:
        return 0.05  # Low arrival rate (1 passenger every 20 minutes on average)

def time_to_string(minutes):
    """Convert minutes from midnight to HH:MM format"""
    hours = int(minutes // 60) % 24
    mins = int(minutes % 60)
    return f"{hours:02d}:{mins:02d}"


In [201]:
class Passenger:
    def __init__(self, pid, pickup, destination, start_time):
        self.id = pid
        self.pickup = pickup
        self.destination = destination
        self.start_time = start_time  # Track when passenger enters system
    
    def __repr__(self, color = '\033[97m'):         # changes representation of passenger to be easier to read
        return f"{color}P{self.id}({self.pickup} --> {self.destination})"

In [202]:
def passenger_generator(env, elevator):
    # Start with pid 1
    pid = 1         
    
    while True:
        # Get current arrival rate based on time of day
        current_rate = get_arrival_rate(env.now)
        
        # Calculate inter-arrival time using exponential distribution
        if current_rate > 0:
            inter_arrival_time = random.expovariate(current_rate)
        else:
            # Default to 1 hour if rate is 0 
            inter_arrival_time = 60  
        
        
        
        pickup = random.randint(0, NUM_FLOORS - 1)           # random floor
        destination = random.randint(0, NUM_FLOORS - 1)       # random destination
        while pickup == destination:
            destination = random.randint(0,NUM_FLOORS -1)
        
        
        start_time = env.now  # Record start time
        passenger = Passenger(pid, pickup, destination, start_time) # create the passenger
        current_time_str = time_to_string(env.now)
        print(f"[{current_time_str}] Passenger {pid} appears at floor {pickup}, wants to go to {destination}")
        elevator.request_pickup(passenger)              # request pickup from elevator
        pid +=1


        yield env.timeout(inter_arrival_time)

In [203]:
class Dispatcher:
    def __init__(self, elevators):
        self.elevators = elevators

    def assign_passenger(self, passenger):
        best_elevator = min(
            self.elevators, key = lambda x: abs(x.floor - passenger.pickup)
        )

        best_elevator.request_pickup(passenger)

In [204]:
class Elevator:
    def __init__(self, env, num_floors, strategy = "nearest"):
        self.env = env
        self.floor = 0          # Start at ground floor (0)
        self.direction = 'up'   # up or down
        self.num_floors = num_floors # total
        self.pickup_requests = {floor: [] for floor in range(num_floors)}
        self.onboard = []
        self.strategy = strategy
        self.stats = {
            'total_passengers': 0,
            'total_wait_time': 0,
            'total_travel_time': 0
        }

    def log(self, message):
        current_time = time_to_string(self.env.now)
        print(f"    [{current_time}] Elevator floor: {self.floor} | {message}")

    def request_pickup(self, passenger):
        self.pickup_requests[passenger.pickup].append(passenger)
        self.log(f"Passenger {passenger.id} requests a pickup at floor {passenger.pickup}")

    def find_longest_waiting_request(self):
        longest_wait_time = -1
        target_floor = None
        for floor, passengers in self.pickup_requests.items():
            for p in passengers:
                wait_time = self.env.now - p.start_time
                if wait_time > longest_wait_time:
                    longest_wait_time = wait_time
                    target_floor = floor
        return target_floor

    def run_elevator(self):
        while True:
            # 1) Off-load passengers at current floor
            offboarding = [p for p in self.onboard if p.destination == self.floor]
            for p in offboarding:
                self.onboard.remove(p)
                travel_time = self.env.now - p.start_time
                self.stats['total_travel_time'] += travel_time
                self.stats['total_passengers'] += 1
                self.log(f"Dropped off {p} (travel time: {travel_time:.1f} min)")

            # 2) On-board passengers at current floor
            boarding = self.pickup_requests[self.floor]
            for p in boarding:
                if len(self.onboard) < MAX_CAPACITY:
                    self.onboard.append(p)
                    wait_time = self.env.now - p.start_time
                    self.stats['total_wait_time'] += wait_time
                    self.log(f"Picked up {p} (wait time: {wait_time:.1f} min)")
                else:
                    self.log(f"Elevator full, {p} must wait")
            
            self.pickup_requests[self.floor].clear()

            if not any(self.pickup_requests.values()) and not self.onboard:
                #self.log("idle")
                yield self.env.timeout(1)
                continue


            # 3) Move logic based on strategy
            if self.strategy == "longest_wait":
                target_floor = self.find_longest_waiting_request()
    
                if target_floor is not None and target_floor != self.floor:
                    if target_floor > self.floor:
                        self.direction = 'up'
                        self.floor += 1
                    elif target_floor < self.floor:
                        self.direction = 'down'
                        self.floor -= 1
                        self.log(f"Moving {self.direction} toward floor {target_floor}")
                else:
                # No passengers waiting → continue in current direction
                    self.log("No pickups pending, continuing current direction")
                    if self.direction == 'up':
                        if self.floor < self.num_floors - 1:
                            self.floor += 1
                        else:
                            self.direction = 'down'
                            self.floor -= 1
                    else:
                        if self.floor > 0:
                            self.floor -= 1
                        else:
                            self.direction = 'up'
                            self.floor += 1
            else:
            # Default nearest (up/down sweep)
                if self.direction == 'up':
                    if self.floor < self.num_floors - 1:
                        self.floor += 1
                    else:
                        self.direction = 'down'
                        self.floor -= 1
                else:
                    if self.floor > 0:
                        self.floor -= 1
                    else:
                        self.direction = 'up'
                        self.floor += 1
            self.log(f"Moving {self.direction} to floor {self.floor}")


# 4) Wait for travel time (e.g. 1 time-unit per floor)
            yield self.env.timeout(.5)

In [205]:
class Elevator_Controller:
    def __init__(self, env, NUM_ELEVATORS, NUM_FLOORS):
        self.env = env
        self.elevators = [Elevator(env, NUM_FLOORS) for _ in range(NUM_ELEVATORS)]
        self.dispatcher = Dispatcher(self.elevators)

    def start(self):
        for elevator in self.elevators:
            self.env.process(elevator.run_elevator())
        

In [206]:
def print_simulation_stats(elevator):
    """Print simulation statistics"""
    print("\n" + "="*50)
    print("SIMULATION STATISTICS")
    print("="*50)
    print(f"Total passengers served: {elevator.stats['total_passengers']}")
    if elevator.stats['total_passengers'] > 0:
        avg_wait_time = elevator.stats['total_wait_time'] / elevator.stats['total_passengers']
        avg_travel_time = elevator.stats['total_travel_time'] / elevator.stats['total_passengers']
        print(f"Average wait time: {avg_wait_time:.1f} minutes")
        print(f"Average travel time: {avg_travel_time:.1f} minutes")
    print(f"Simulation duration: {SIM_TIME} minutes ({SIM_TIME/60:.1f} hours)")
    print(f"total passengers {[elevator.stats['total_passengers']]}")
    print("="*50)

In [207]:
# Run the simulation
if __name__ == "__main__":
    random.seed(23)
    env = simpy.Environment()
    elevator = Elevator(env, num_floors=NUM_FLOORS, strategy="longest_wait")  # or "nearest"

    
    # Start both processes
    env.process(passenger_generator(env, elevator))
    env.process(elevator.run_elevator())
    
    print(f"Starting 24-hour elevator simulation...")
    print(f"Simulation will run for {SIM_TIME} minutes ({SIM_TIME/60:.1f} hours)")
    print("-" * 50)
    
    env.run(until=SIM_TIME)
    
    print_simulation_stats(elevator) 

Starting 24-hour elevator simulation...
Simulation will run for 1440 minutes (24.0 hours)
--------------------------------------------------
[00:00] Passenger 1 appears at floor 4, wants to go to 1
    [00:00] Elevator floor: 0 | Passenger 1 requests a pickup at floor 4
    [00:00] Elevator floor: 1 | Moving up to floor 1
    [00:00] Elevator floor: 2 | Moving up to floor 2
    [00:01] Elevator floor: 3 | Moving up to floor 3
    [00:01] Elevator floor: 4 | Moving up to floor 4
    [00:02] Elevator floor: 4 | Picked up P1(4 --> 1) (wait time: 2.0 min)
    [00:02] Elevator floor: 4 | No pickups pending, continuing current direction
    [00:02] Elevator floor: 5 | Moving up to floor 5
    [00:02] Elevator floor: 5 | No pickups pending, continuing current direction
    [00:02] Elevator floor: 6 | Moving up to floor 6
    [00:03] Elevator floor: 6 | No pickups pending, continuing current direction
    [00:03] Elevator floor: 7 | Moving up to floor 7
    [00:03] Elevator floor: 7 | No picku